## Ant Colony Optimization Algorithm 
#### *Object Replacement and Update Pheromones Variant*
An object CAN'T be selected twice in a single partial solution

In [329]:
import numpy as np
from anpy.Ant import Ant
import pandas as pd
from anpy.AntFactory import AntFactory
from anpy.support import random_selection
from anpy.Neighborhood import Neighborhood

pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 1500)
pd.set_option('display.width', 1000)

# Actual Execution has to be with:
# 10 ANTS | 100 ITERATIONS
ITERATIONS = 100
ANTS_NUMS = 10
KNAPSACK_CAPACITY = 750 # Total Knapsack load capacity

best_ant_iteration = pd.DataFrame(columns=
    ['Iteration',
    'Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])
worst_ant_iteration = pd.DataFrame(columns=
    ['Iteration',
    'Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])
best_ant_run = pd.DataFrame(columns=
    ['Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])
worst_ant_run = pd.DataFrame(columns=
    ['Ant ID',
    'Partial Solution (S)',
    'Profit (Z)',
    'Capacity (V)'])

In [330]:
def setup_neighborhood():
    # Create Neighborhood
    N1 = Neighborhood(ANTS_NUMS)
    
    # Set Neighborhood's Weights & Profits
    N1.set_weights("weights.txt")
    N1.set_profits("profits.txt")
    N = N1.generate_neighborhood()
    N1.set_neighborhood(N)
    return N1

def get_best_ant_iter(iteration:int, ant:Ant):
    best_ant_iteration.loc[iteration] = [iteration,
        ant.id,                
        ant.s,
        ant.z,
        np.sum(ant.s)]
def get_worst_ant_iter(iteration:int, ant:Ant):
    worst_ant_iteration.loc[iteration] = [iteration,
        ant.id,                
        ant.s,
        ant.z,
        np.sum(ant.s)]

In [331]:
def aco():
    global_best_ant = AntFactory.create(0)
    N = setup_neighborhood()
    for iter in range(1, ITERATIONS+1):
        iter_best_ant = AntFactory.create(0)
        i = 0
        N.set_ants(ANTS_NUMS)     
        while i < len(N.ants):
            random_ant = N.ants[np.random.randint(0,len(N.ants))]
            Vc = KNAPSACK_CAPACITY; 
            N.set_neighborhood(N.current_N)
            while not random_ant.hasWorked:
                available_objects = N.get_available_objects(Vc)
                if len(available_objects) == 0: random_ant.hasWorked = True; i += 1; break
                objects_probs = N.get_object_probs(available_objects)
                object = random_selection(available_objects, objects_probs)
                random_ant.add_object(object)
                N.update_N(object)
                Vc -= object.weight
            if random_ant.z > iter_best_ant.z: iter_best_ant = random_ant
        if iter_best_ant.z > global_best_ant.z: global_best_ant = iter_best_ant
        ants_profits = [ant.z for ant in N.ants]    
        get_best_ant_iter(iter, N.ants[np.argmax(ants_profits)])
        get_worst_ant_iter(iter, N.ants[np.argmin(ants_profits)])
        N.evaporation_mechanism()
        N.set_neighborhood(N.update_pheromones(random_ant, global_best_ant))
    return global_best_ant      


In [332]:
best_ant = aco()
best_ant_data = pd.DataFrame(columns=['Ant ID', 'S (Solution)', 'Z (Profit)', 'V (Capacity)'])
best_ant_data.loc['1'] = [best_ant.id, best_ant.s, best_ant.z, sum(best_ant.s)]
print("\033[1mBest Ant From Execution:\033[0m\n \tITERATIONS: 100 | ANTS: 10")
best_ant_data

Best Ant From Execution:
 	ITERATIONS: 100 | ANTS: 10


,Ant ID,S (Solution),Z (Profit),V (Capacity)
1,3,"[90, 77, 98, 80, 94, 120, 118, 70]",1452,747


In [333]:
# Display Iteration's Best Ant
# best_ant_iteration.set_index('Iteration', inplace=True)
# best_ant_iteration

In [334]:
# Display Iteration's Worst Ant
# worst_ant_iteration.set_index('Iteration', inplace=True)
# worst_ant_iteration

In [335]:
# Get Run's Best Ant
best_ant_run.loc[1] = best_ant_iteration.loc[np.argmax(best_ant_iteration['Profit (Z)'])+1]
best_ant_run

,Ant ID,Partial Solution (S),Profit (Z),Capacity (V)
1,3,"[90, 77, 98, 80, 94, 120, 118, 70]",1452,747


In [336]:
# Get Run's Worst Ant
worst_ant_run.loc[1] = worst_ant_iteration.loc[np.argmin(worst_ant_iteration['Profit (Z)'])+1]
worst_ant_run

,Ant ID,Partial Solution (S),Profit (Z),Capacity (V)
1,6,"[73, 82, 87, 77, 94, 70, 90, 80]",1249,653
